In [0]:
# #@test {"output": "ignore"}

# print 'Installing dependencies...'
# !apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
# !pip install -qU pyfluidsynth pretty_midi

# !pip install -qU magenta

# # Hack to allow python to pick up the newly-installed fluidsynth lib. 
# # This is only needed for the hosted Colab environment.
# import ctypes.util
# orig_ctypes_util_find_library = ctypes.util.find_library
# def proxy_find_library(lib):
#   if lib == 'fluidsynth':
#     return 'libfluidsynth.so.1'
#   else:
#     return orig_ctypes_util_find_library(lib)
# ctypes.util.find_library = proxy_find_library

# print 'Importing libraries and defining some helper functions...'
# from google.colab import files

# import magenta.music as mm
# import magenta
# import tensorflow

# print '🎉 Done!'
# print magenta.__version__ 
# print tensorflow.__version__

In [1]:
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print '🎉 Done!'
print magenta.__version__ 
print tensorflow.__version__


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

🎉 Done!
0.3.19
1.13.1


In [0]:
import numpy as np

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = "/content/gdrive/My Drive/Applied Deep Learning/Outputs/"

In [0]:
new_songs_raw = np.load(data_path + 'new_songs.npy')

In [0]:
new_songs_raw.shape

## Generate MIDI


In [10]:
from music21 import instrument, note, stream, chord

 
Music21 v.4 is the last version that will support Python 2.
Please start using Python 3 instead.

Set music21.environment.UserSettings()['warnings'] = 0
to disable this message.



In [0]:
def create_midi_irish(prediction_output, number, output_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        current_note = pattern[0]
        velocity = pattern[1]
        duration = pattern[2]
        
        notes = []
        new_note = note.Note(int(current_note))
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)   
            
        offset += duration

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=output_path+'irish_output%d.mid' % number)

In [0]:
output_path = "/content/gdrive/My Drive/Applied Deep Learning/Irish Output/"

In [0]:
for k in range(new_songs_raw.shape[0]):
  create_midi_irish(new_songs_raw[k], k, output_path)

## Play Music Using Magenta Player (Deprecated)

In [0]:
from magenta.protobuf import music_pb2

magenta_songs = []

for k in range(new_songs_raw.shape[0]):
  song = music_pb2.NoteSequence()

  st_time = 0
  end_time = 0
  
  for m in range(new_songs_raw.shape[1]):
    # Add the notes to the sequence.
    
    pitch = int(new_songs_raw[k,m,0])
    st_time = end_time
    end_time = st_time + new_songs_raw[k,m,2]
    veloc = int(new_songs_raw[k,m,1])
    
    song.notes.add(pitch=pitch, 
                   start_time=st_time, 
                   end_time=end_time, 
                   velocity=veloc)
  
  magenta_songs.append(song)


In [0]:
for k in range(len(magenta_songs)):
  magenta_songs[k].total_time = 120
  magenta_songs[k].tempos.add(qpm=100);
  # This is a colab utility method that visualizes a NoteSequence.
  mm.plot_sequence(magenta_songs[k])

  # This is a colab utility method that plays a NoteSequence.
  # mm.play_sequence(magenta_songs[k],synth=mm.fluidsynth)
  mm.play_sequence(magenta_songs[k],synth=mm.synthesize)


Loading BokehJS ...

In [0]:
# """ This module prepares midi file data and feeds it to the neural
#     network for training """
# import glob
# import pickle
# import numpy
# from music21 import converter, instrument, note, chord
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from keras.layers import Activation
# from keras.utils import np_utils
# from keras.callbacks import ModelCheckpoint

In [0]:
# from music21 import converter, instrument, note, chord
# notes = []
# for file in glob.glob("*.mid"):
#     midi = converter.parse(file)
#     notes_to_parse = None
#     parts = instrument.partitionByInstrument(midi)
#     if parts: # file has instrument parts
#         notes_to_parse = parts.parts[0].recurse()
#     else: # file has notes in a flat structure
#         notes_to_parse = midi.flat.notes
#     for element in notes_to_parse:
#         if isinstance(element, note.Note):
#             notes.append(str(element.pitch))
#         elif isinstance(element, chord.Chord):
#             notes.append('.'.join(str(n) for n in element.normalOrder))

In [0]:
# notes